In [3]:
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/users/jamd1/Isca/')
from isca_tools import cesm
from isca_tools.utils import area_weighting, lat_lon_rolling
from isca_tools.utils import print_ds_var_list
from isca_tools.papers.byrne_2021 import get_quant_ind
from isca_tools.utils.moist_physics import moist_static_energy, sphum_sat
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import matplotlib.patches as patches
from tqdm.notebook import tqdm
import matplotlib
from matplotlib.dates import ConciseDateConverter
matplotlib.units.registry[cftime.DatetimeNoLeap] = ConciseDateConverter()  # to allow plotting time as x axis
# plt.style.use('/home/users/jamd1/Isca/jobs/publish.mplstyle')
import os
print(os.getcwd())

/home/users/jamd1/Isca


In [27]:
# Basic info for getting data on model levels
dir_era5 = '/badc/ecmwf-era5/data/'
id_dataset = 'ecmwf-era5'
id_data = 'data'
id_output = 'oper'      # operatonal as opposed to enda - ensemble data assimilation
id_analysis = 'an'      # use analysis as opposed to forecast
id_level = 'ml'        # use model levels as opposed to surface

In [49]:
year = 1979
month = 1
day = 1
hour = 2      # 2 AM
id_var = 't'        # temperature

In [50]:
def get_file_path(id_var, hour, year, month, day, id_dataset='ecmwf-era5', id_data='data', id_output='oper', id_analysis='an', id_level='ml'):
    dir = f"/badc/{id_dataset}/{id_data}/{id_output}/{id_analysis}_{id_level}/{year}/{month:02d}/{day:02d}"
    file_name = f"{id_dataset}_{id_output}_{id_analysis}_{id_level}_{year}{month:02d}{day:02d}{hour:02d}00.{id_var}.nc"
    return f"{dir}/{file_name}"

In [51]:
ds = xr.load_dataset(get_file_path(id_var, hour, year, month, day))

In [52]:
ds.time

<xarray.DataArray 'time' (time: 1)> Size: 8B
array(['1979-01-01T02:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 8B 1979-01-01T02:00:00
Attributes:
    long_name:  time